In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time

In [2]:
url = "https://m.imdb.com/chart/top/"

In [3]:
page = requests.get(url,headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36' })

In [4]:
soup = BeautifulSoup(page.content, "html.parser")

In [5]:
scraped_movies = soup.find_all('h3', class_='ipc-title__text')

In [6]:
movies = []
for movie in scraped_movies:
    parts = movie.text.split(' ', 1)
    if len(parts) == 2:
        movies.append(parts[1])
del movies[0]
movies =movies [:-10]

In [7]:
scraped_ratings = soup.find_all('span', class_='ipc-rating-star ipc-rating-star--base ipc-rating-star--imdb ratingGroup--imdb-rating')

In [8]:
ratings=[]
for rating in scraped_ratings:
    ratings.append(rating['aria-label'].split(': ')[1])

In [9]:
scraped_year = soup.find_all('span', class_='sc-b189961a-8 kLaxqf cli-title-metadata-item')

In [10]:
years = []
for year in scraped_year:
    if year.text.isdigit():
        years.append(year.text)

In [11]:
scraped_images = soup.find_all('img', class_='ipc-image')

In [12]:
images=[]
for image in scraped_images :
    images.append(image.get('src'))
images =images [:-1]

In [13]:
scraped_all_info = soup.find_all('a', class_='ipc-lockup-overlay ipc-focusable')

In [22]:
list_genres =[]
list_directors =[]
list_stars=[]
for button in scraped_all_info:
      url = button.get('href')
      if not url.startswith('http'):
          url = 'https://m.imdb.com/' + url
          page = requests.get(url, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36'})
          soup = BeautifulSoup(page.content, "html.parser")
          genres=[]
          for genre in soup.find_all('span', class_='ipc-chip__text'):
              genres.append(genre.text.strip())
          genres =genres[:-1]
          list_genres.append(genres)
          element = soup.find('li', class_='ipc-metadata-list__item')
          director=element.find('a', class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link')
          director = director.text.strip()
          list_directors.append(director)
          for human in soup.find_all('a', class_='ipc-metadata-list-item__label ipc-metadata-list-item__label--link'):
                  if human.text.strip() == "Stars":
                    url1 = human.get('href')
                    if not url1.startswith('http'):
                      url1 = 'https://m.imdb.com/' + url1
                      page1 = requests.get(url1, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36'})
                      soup1 = BeautifulSoup(page1.content, "html.parser")
                      stars=[]
                      for star in soup1.find_all('h4'):
                          stars.append(star.text.strip())
          list_stars.append(stars)

In [23]:
data = pd.DataFrame()
data['Movie Names'] = movies
data['Ratings'] = ratings
data['Release year'] = years
data['Genres'] = list_genres
data['Image'] = images
data['Stars'] = list_stars
data['Directors'] = list_directors
data.head()
data.to_csv('IMDB Top Movies.csv', index=False)
pd.read_csv("IMDB Top Movies.csv")

,Movie Names,Ratings,Release year,Genres,Image,Stars,Directors
0,The Shawshank Redemption,9.3,1994,['Drama'],https://m.media-amazon.com/images/M/MV5BNDE3OD...,"['Tim Robbins', 'Morgan Freeman', 'Bob Gunton'...",Frank Darabont
1,The Godfather,9.2,1972,"['Crime', 'Drama']",https://m.media-amazon.com/images/M/MV5BM2MyNj...,"['Marlon Brando', 'Al Pacino', 'James Caan', '...",Francis Ford Coppola
2,The Dark Knight,9.0,2008,"['Action', 'Crime', 'Drama']",https://m.media-amazon.com/images/M/MV5BMTMxNT...,"['Christian Bale', 'Heath Ledger', 'Aaron Eckh...",Christopher Nolan
3,The Godfather Part II,9.0,1974,"['Crime', 'Drama']",https://m.media-amazon.com/images/M/MV5BMWMwMG...,"['Al Pacino', 'Robert Duvall', 'Diane Keaton',...",Francis Ford Coppola
4,12 Angry Men,9.0,1957,"['Crime', 'Drama']",https://m.media-amazon.com/images/M/MV5BMWU4N2...,"['Martin Balsam', 'John Fiedler', 'Lee J. Cobb...",Sidney Lumet
...,...,...,...,...,...,...,...
245,It Happened One Night,8.1,1934,"['Comedy', 'Romance']",https://m.media-amazon.com/images/M/MV5BMDg5Mm...,"['Clark Gable', 'Claudette Colbert', 'Walter C...",Frank Capra
246,Aladdin,8.0,1992,"['Animation', 'Adventure', 'Comedy']",https://m.media-amazon.com/images/M/MV5BZTg5ZT...,"['Scott Weinger', 'Robin Williams', 'Linda Lar...",Ron Clements
247,Dances with Wolves,8.0,1990,"['Adventure', 'Drama', 'Western']",https://m.media-amazon.com/images/M/MV5BMTY3OT...,"['Kevin Costner', 'Mary McDonnell', 'Graham Gr...",Kevin Costner
248,Drishyam,8.2,2015,"['Crime', 'Drama', 'Mystery']",https://m.media-amazon.com/images/M/MV5BYmJhZm...,"['Ajay Devgn', 'Shriya Saran', 'Tabu', 'Rajat ...",Nishikant Kamat


In [37]:
list_genres_cleaned=[]
for genre in list_genres:
  list_genres_cleaned.append(str(genre).replace('[','').replace(']','').replace("'",'').replace("'",''))

In [41]:
list_stars_cleaned=[]
for star in list_stars:
  list_stars_cleaned.append(str(star).replace('[','').replace(']','').replace("'",'').replace("'",''))

In [42]:
ata = pd.DataFrame()
data['Movie Names'] = movies
data['Ratings'] = ratings
data['Release year'] = years
data['Genres'] = list_genres_cleaned
data['Image'] = images
data['Stars'] = list_stars_cleaned
data['Directors'] = list_directors
data.head()
data.to_csv('IMDB Top Movies cleaned.csv', index=False)
pd.read_csv("IMDB Top Movies cleaned.csv")

,Movie Names,Ratings,Release year,Genres,Image,Stars,Directors
0,The Shawshank Redemption,9.3,1994,Drama,https://m.media-amazon.com/images/M/MV5BNDE3OD...,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Frank Darabont
1,The Godfather,9.2,1972,"Crime, Drama",https://m.media-amazon.com/images/M/MV5BM2MyNj...,"Marlon Brando, Al Pacino, James Caan, Richard ...",Francis Ford Coppola
2,The Dark Knight,9.0,2008,"Action, Crime, Drama",https://m.media-amazon.com/images/M/MV5BMTMxNT...,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",Christopher Nolan
3,The Godfather Part II,9.0,1974,"Crime, Drama",https://m.media-amazon.com/images/M/MV5BMWMwMG...,"Al Pacino, Robert Duvall, Diane Keaton, Robert...",Francis Ford Coppola
4,12 Angry Men,9.0,1957,"Crime, Drama",https://m.media-amazon.com/images/M/MV5BMWU4N2...,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",Sidney Lumet
...,...,...,...,...,...,...,...
245,It Happened One Night,8.1,1934,"Comedy, Romance",https://m.media-amazon.com/images/M/MV5BMDg5Mm...,"Clark Gable, Claudette Colbert, Walter Connoll...",Frank Capra
246,Aladdin,8.0,1992,"Animation, Adventure, Comedy",https://m.media-amazon.com/images/M/MV5BZTg5ZT...,"Scott Weinger, Robin Williams, Linda Larkin, J...",Ron Clements
247,Dances with Wolves,8.0,1990,"Adventure, Drama, Western",https://m.media-amazon.com/images/M/MV5BMTY3OT...,"Kevin Costner, Mary McDonnell, Graham Greene, ...",Kevin Costner
248,Drishyam,8.2,2015,"Crime, Drama, Mystery",https://m.media-amazon.com/images/M/MV5BYmJhZm...,"Ajay Devgn, Shriya Saran, Tabu, Rajat Kapoor, ...",Nishikant Kamat
